In [14]:
!pip install textblob
!pip install vaderSentiment nltk nrclex
!pip install nrclex
!pip install -U scikit-learn
from google.colab import drive
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # Import OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.compose import ColumnTransformer # Import ColumnTransformer
from textblob import TextBlob
from nrclex import NRCLex
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # For VADER
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
# Download the necessary NLTK data
nltk.download('punkt_tab') # Downloading the missing punkt_tab resource.


drive.mount('/content/drive',force_remount=True)
# Add the 'libs' directory to the system path
drive_libs_path = '/content/drive/MyDrive/libs'
sys.path.append(drive_libs_path)

from usage_examples import get_acceptance_indexes, CombinedAcceptanceIndexer
from noise_filter import NoiseFilter

def mount_drive():
    drive.mount('/content/drive', force_remount=True)


def get_sentiment(text, method='textblob'):
    if method == 'textblob':
        blob = TextBlob(text)
        return blob.sentiment.polarity, blob.sentiment.subjectivity
    elif method == 'vader':
        analyzer = SentimentIntensityAnalyzer()
        scores = analyzer.polarity_scores(text)
        return scores['compound'], scores['compound']
    elif method == 'nrc':
        emotion_analyzer = NRCLex(text)
        sentiment_scores = emotion_analyzer.affect_frequencies
        polarity = sentiment_scores.get('positive', 0) - sentiment_scores.get('negative', 0)
        subjectivity = sum(sentiment_scores.values()) - sentiment_scores.get('anticipation', 0) - sentiment_scores.get('trust', 0)
        return polarity, subjectivity
    else:
        raise ValueError(f"Invalid method: {method}. Choose from 'textblob', 'vader', or 'nrc'.")

def process_comments(row, method='textblob'):
    try:
        comment = row['comments']  # Get the 'comments' string directly
    except AttributeError:
        return 0, 0  # Handle cases where 'comments' column is missing or invalid

    # Call get_sentiment with the specified method
    polarity, subjectivity = get_sentiment(str(comment), method=method)

    return polarity, subjectivity

def load_data(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    df.rename(columns={'comments_orig': 'comments'}, inplace=True)

    # Standard preprocessing\n",
    df['clean_title'] = df['clean_title'].astype(str).str.lower()
    df['comments'] = df['comments'].astype(str)
    df['separated_comment'] = ''

    return df


def preprocess_data(df,method='textblob'):
    df['separated_comment'] = df['comments'].apply(lambda x: x.split('|__|'))
    df['comments_polarity'], df['comments_subjectivity'] = zip(*df.apply(lambda row: process_comments(row, method=method), axis=1))
    df['clean_title_polarity'], df['clean_title_subjectivity'] = zip(*df['clean_title'].apply(lambda title: get_sentiment(title, method=method)))
    return df

def prepare_features_and_labels(df,method='textblob',base_feature=False):
    # Select features for the model
    base_features = ['clean_title', 'score', 'separated_comment', 'num_comments', 'upvote_ratio','comments_polarity', 'comments_subjectivity', 'clean_title_polarity',
            'clean_title_subjectivity']
    base_numerical_features = ['score', 'num_comments', 'upvote_ratio','comments_polarity', 'comments_subjectivity', 'clean_title_polarity',
            'clean_title_subjectivity']
    if base_feature:  # If base_feature is True, select only base features
        features = base_features
        numerical_features = base_numerical_features
    else:  # If base_feature is False, select base features + method-specific feature
        if method == 'textblob':
            features = base_features + ['TextBlobAcceptanceIndexer']
            numerical_features = base_numerical_features + ['TextBlobAcceptanceIndexer']
        elif method == 'nrc':
            features = base_features + ['NRCAcceptanceIndexer']
            numerical_features = base_numerical_features + ['NRCAcceptanceIndexer']
        elif method == 'vader':
            features = base_features + ['VADERAcceptanceIndexer']
            numerical_features = base_numerical_features + ['VADERAcceptanceIndexer']
        else:
            raise ValueError(f"Invalid method: {method}. Choose from 'textblob', 'vader', or 'nrc'.")

    X = df[features]

	# *** Handle missing values (NaNs) *** Impute numerical features first
    X[numerical_features] = X[numerical_features].fillna(X[numerical_features].mean())


    # Target variable
    y = df['2_way_label'].astype(str)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)


    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Create a ColumnTransformer to apply TF-IDF to text data,
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numerical_features),
            ('text', TfidfVectorizer(max_features=5000), 'clean_title'),
        ])

    # Fit and transform the training data
    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    # *** Clip negative values to 0 ***
    # This ensures all values passed to MultinomialNB are non-negative
    X_train = X_train.toarray()  # Convert to dense array if necessary
    X_test = X_test.toarray()    # Convert to dense array if necessary

    X_train[X_train < 0] = 0
    X_test[X_test < 0] = 0

    return (X_train, X_test, y_train, y_test), label_encoder, preprocessor # Return preprocessor


def build_pipeline():
    # No changes here, the pipeline uses the preprocessor now
    return Pipeline([
        ('classifier', MultinomialNB())
    ])

def evaluate_model(model, X_test, y_test, label_encoder):
    y_pred = model.predict(X_test)
    print("Classification Report:")
    unique_labels = np.unique(y_test)
    target_names = label_encoder.classes_[unique_labels]
    print(classification_report(y_test, y_pred, target_names=target_names))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")

def predict_new_comments(model, label_encoder, preprocessor, comments, title, method='textblob', base_feature=False):
    # Define base features
    base_features_df = pd.DataFrame({'clean_title': title * len(comments),
                                       'score': [0] * len(comments),
                                       'num_comments': [0] * len(comments),
                                       'upvote_ratio': [0] * len(comments),
                                       'separated_comment': [''] * len(comments),
                                       'comments_polarity': [0] * len(comments),  # Add comments_polarity
                                       'comments_subjectivity': [0] * len(comments),  # Add comments_subjectivity
                                       'clean_title_polarity': [0] * len(comments),  # Add clean_title_polarity
                                       'clean_title_subjectivity': [0] * len(comments)})

    # Add method-specific features if base_feature is False
    if not base_feature:
        if method == 'textblob':
            base_features_df['TextBlobAcceptanceIndexer'] = [0] * len(comments)
        elif method == 'vader':
            base_features_df['VADERAcceptanceIndexer'] = [0] * len(comments)
        elif method == 'nrc':
            base_features_df['NRCAcceptanceIndexer'] = [0] * len(comments)
        else:
            raise ValueError(f"Invalid method: {method}. Choose from 'textblob', 'vader', or 'nrc'.")

   # Preprocess the new comments using the same preprocessor
    comments_transformed = preprocessor.transform(base_features_df)
    # Make predictions
    predictions = model.predict(comments_transformed)
    return label_encoder.inverse_transform(predictions)

def main():
    mount_drive()
    file_path = "/content/drive/My Drive/datasets/data_with_indexers_applied_to_original_data_31_03_2025.csv"
    df = preprocess_data(load_data(file_path),method='nrc')
    (X_train, X_test, y_train, y_test), label_encoder, preprocessor = prepare_features_and_labels(df,method='nrc',base_feature=False) # Get preprocessor
    model = build_pipeline()
    model.fit(X_train, y_train)
    evaluate_model(model, X_test, y_test, label_encoder)
    new_title = "Shocking Discovery Leaves Internet Divided—Hoax or Breakthrough?"
    new_comments = ["This is outrageous! There’s no way this can be true!", "I can’t believe people are falling for this. It’s just a hoax."]
    # Pass the preprocessor to predict_new_comments
    print("Predictions for new comments:", predict_new_comments(model, label_encoder, preprocessor, new_comments, new_title,method='nrc', base_feature=False))

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Mounted at /content/drive
Mounted at /content/drive


<ipython-input-14-c5d6323699fe>:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numerical_features] = X[numerical_features].fillna(X[numerical_features].mean())


Classification Report:
              precision    recall  f1-score   support

           0       0.27      0.92      0.42      3317
           1       0.91      0.24      0.38     10842

    accuracy                           0.40     14159
   macro avg       0.59      0.58      0.40     14159
weighted avg       0.76      0.40      0.39     14159

Confusion Matrix:
[[3059  258]
 [8253 2589]]
Test Accuracy: 0.39889822727593754
Predictions for new comments: ['0' '0']
